# CSCI 3155 : Homework 7



__Name__:

Qiuyang Fu

In [1]:
// TEST HELPER: Do not EDIT this cell please.
def passed(points: Int) {
    require(points >=0)
    if (points == 1) print(s"\n*** Tests Passed (1 point) ***\n")
    else print(s"\n*** Tests Passed ($points points) ***\n")
}

defined function passed

## Problem 1: Functions to Lettuce

### Problem 1a (4 points)

Extend the abstract syntax of lettuce to include functions

$$\begin{array}{rcll}
\mathbf{Expr} & \rightarrow & \\
& ...\\
 & | & FunDef( \mathbf{Identifier}, \mathbf{Expr}) & \texttt{function (arg) body_expr} \\ 
 & | & FunCall(\mathbf{Expr}, \mathbf{Expr}) & \texttt{fun_expr(arg_expr)} \\
\end{array}$$

In [2]:
sealed trait Expr
case class  Const(n : Int) extends Expr
case class  Ident(s : String) extends Expr
case object True extends Expr
case object False extends Expr
case class  Plus( e1 : Expr, e2 : Expr) extends Expr
case class  Minus( e1 : Expr, e2 : Expr) extends Expr
case class  Mult( e1 : Expr, e2 : Expr) extends Expr
case class  Neg( e1 : Expr) extends Expr
case class  And( e1 : Expr, e2 : Expr) extends Expr
case class  Or( e1 : Expr, e2 : Expr) extends Expr
case class  Geq( e1 : Expr, e2 : Expr) extends Expr
case class  Eq( e1 : Expr, e2 : Expr) extends Expr
case class  ITE( ec : Expr, et : Expr, ef : Expr) extends Expr
case class  Let( name : String, ed : Expr, ei : Expr) extends Expr
case class  FunDef( id : String, e : Expr) extends Expr
case class  FunCall( func : Expr, arg : Expr) extends Expr

defined trait Expr
defined class Const
defined class Ident
defined object True
defined object False
defined class Plus
defined class Minus
defined class Mult
defined class Neg
defined class And
defined class Or
defined class Geq
defined class Eq
defined class ITE
defined class Let
defined class FunDef
defined class FunCall

In [3]:
val e1 = FunDef( "x", Plus(Ident("x"), Const(1)))
val e2 = FunCall( Ident("f"), Const(1))
val e3 = Let( "f", 
              FunDef( "x", Plus(Ident("x"), Const(1))), 
              FunCall( Ident("f"), Const(1)))
passed(4)


*** Tests Passed (4 points) ***


e1: FunDef = FunDef("x", Plus(Ident("x"), Const(1)))
e2: FunCall = FunCall(Ident("f"), Const(1))
e3: Let = Let(
  "f",
  FunDef("x", Plus(Ident("x"), Const(1))),
  FunCall(Ident("f"), Const(1))
)

### Problem 1b (4 points)
Recall that our values can be of the form:
- Real numbers: $\mathbb{R}$,
- Booleans: $\mathbb{B}$,
- Closures: $\mathbb{C}$ of the form $\text{Closure}(x, \texttt{e}, \sigma)$,
- Error $\mathbf{error}$.

Extends the defintion of Values to include closures

In [4]:
type Environment = 
    Map[String, Value]

sealed trait Value
case class NumVal(x : Int) extends Value
case class BinVal(x : Boolean) extends Value
case class Closure(id: String, e: Expr, env: Environment) extends Value
case object Error extends Value

defined type Environment
defined trait Value
defined class NumVal
defined class BinVal
defined class Closure
defined object Error

In [5]:
val v1 : Value = Closure( "x", Plus(Ident("x"), Const(1)), Map())
val v2 : Value = Closure( "y", Plus(Ident("x"), Ident("y")), Map( "x" -> NumVal(1)))
passed(4)


*** Tests Passed (4 points) ***


v1: Value = Closure("x", Plus(Ident("x"), Const(1)), Map())
v2: Value = Closure("y", Plus(Ident("x"), Ident("y")), Map("x" -> NumVal(1)))

### Problem 1c (20 points)

Extend the eval function to be able to handle non recursive functions.

Recall the inference rules to define the operational semantics of functions in Lettuce:

$$ \begin{array}{c}
 \\
\hline
\sigma \vdash \texttt{FunDef}\ x\ e_b \Downarrow \texttt{Closure}(x, e_b, \sigma) \\
\end{array} \text{(FunDef)} $$

$$ \begin{array}{c}
\sigma \vdash e_f \Downarrow \texttt{Closure}(x, e_b, \pi) \qquad
\sigma \vdash e_a \Downarrow v_a \qquad
\pi[x\rightarrow v_a] \vdash e_b \Downarrow v_r\\
\hline
\sigma \vdash \texttt{FunCall}\ e_f\ e_a \Downarrow v_r \\
\end{array} \text{(FunCall)} $$

In [6]:
def eval( e : Expr, env : Environment) : Value = 
    e match {
        case Const(n)        => NumVal(n) 
        case True            => BinVal(true)
        case False           => BinVal(false)
        case Ident(s)        => if (env.contains(s)) env(s) else Error
        case Plus(e1, e2)    => (eval(e1,env), eval(e2,env)) match {
            case (NumVal(v1), NumVal(v2)) => NumVal( v1 + v2 )
            case _ => Error
        }
        case Minus(e1, e2)   => (eval(e1,env), eval(e2,env)) match {
            case (NumVal(v1), NumVal(v2)) => NumVal( v1 - v2 )
            case _ => Error
        }
        case Mult(e1, e2)    => (eval(e1,env), eval(e2,env)) match {
            case (NumVal(v1), NumVal(v2)) => NumVal( v1 * v2 )
            case _ => Error
        }
        case Neg(e1)         => eval(e1, env) match {
            case NumVal(v) => NumVal(-v)
            case BinVal(v) => BinVal(!v)
            case _ => Error
        }
        case And(e1, e2)     => (eval(e1,env), eval(e2,env)) match {
            case (BinVal(v1), BinVal(v2)) => BinVal( v1 && v2 )
            case _ => Error
        }
        case Or(e1, e2)      => (eval(e1,env), eval(e2,env)) match {
            case (BinVal(v1), BinVal(v2)) => BinVal( v1 || v2 )
            case _ => Error
        }
        case Geq(e1, e2)     => (eval(e1,env), eval(e2,env)) match {
            case (NumVal(v1), NumVal(v2)) => BinVal( v1 >= v2 )
            case _ => Error
        }
        case Eq(e1, e2)      => (eval(e1,env), eval(e2,env)) match {
            case (NumVal(v1), NumVal(v2)) => BinVal( v1 == v2 )
            case (BinVal(v1), BinVal(v2)) => BinVal( v1 == v2 )
            case (BinVal(_), NumVal(_)) => BinVal(false)
            case (NumVal(_), BinVal(_)) => BinVal(false)
            case _ => Error
        }
        case ITE(ec, et, ef) => eval(ec,env) match {
            case BinVal(true) => eval(et,env)
            case BinVal(false) => eval(ef,env)
            case _ => Error
        }
        case Let(name, ed, ei) => eval(ed, env) match {
            case Error => Error
            case vd => {
                val newenv = env + ( name -> vd)
                eval(ei, newenv)
            }
        }
        case FunDef(name, e) => Closure(name, e, env)
        case FunCall(func, arg) => eval(func, env) match {
              case Closure(name, e, env) =>
                val newenv = env + (name -> eval(arg, env))
                eval(e, newenv)
              case BinVal(e) => BinVal(e)
              case NumVal(e) => NumVal(e)
              case Error => Error
        }
    }

defined function eval

In [7]:
val e1 : Expr = Const(10)
val e2 : Expr = True
val e3 : Expr = Plus( Const(1), Const(2))
val e4 : Expr = And( Geq(e1, e3), e2 )
val e5 : Expr = Let( "x", Const(10), Mult(Ident("x"), Ident("x")))
val emptyenv : Environment = Map()
assert( eval( e1, emptyenv) == NumVal(10) , "test 1")
assert( eval( e2, emptyenv) == BinVal(true), "test 2")
assert( eval( e3, emptyenv) == NumVal(3), "test 3")
assert( eval( e4, emptyenv) == BinVal(true), "test 4")
assert( eval( e5, emptyenv) == NumVal(100), "test 5")
passed(3)


*** Tests Passed (3 points) ***


e1: Expr = Const(10)
e2: Expr = True
e3: Expr = Plus(Const(1), Const(2))
e4: Expr = And(Geq(Const(10), Plus(Const(1), Const(2))), True)
e5: Expr = Let("x", Const(10), Mult(Ident("x"), Ident("x")))
emptyenv: Environment = Map()

In [8]:
val e1 : Expr = FunDef("x", Const(10))
val e2 : Expr = FunDef("y", And(Ident("x"), Ident("y")))
val emptyenv : Environment = Map()
assert( eval( e1, emptyenv) == Closure("x", Const(10), emptyenv), "test 6")
assert( eval( e2, emptyenv) == Closure("y", And(Ident("x"), Ident("y")), emptyenv), "test 7")
passed(3)


*** Tests Passed (3 points) ***


e1: Expr = FunDef("x", Const(10))
e2: Expr = FunDef("y", And(Ident("x"), Ident("y")))
emptyenv: Environment = Map()

In [9]:
val e1 : Expr = FunCall( FunDef("x", Const(10)), True)
val e2 : Expr = FunCall( FunDef("y", And(Ident("x"), Ident("y"))), False)
val emptyenv : Environment = Map()
assert( eval( e1, emptyenv) == NumVal(10), "test 1")
assert( eval( e2, emptyenv) == Error, "test 2")
passed(3)


*** Tests Passed (3 points) ***


e1: Expr = FunCall(FunDef("x", Const(10)), True)
e2: Expr = FunCall(FunDef("y", And(Ident("x"), Ident("y"))), False)
emptyenv: Environment = Map()

In [10]:
val e1 : Expr = FunCall( FunDef("y", Mult( Ident("y"), Const(2))), Const(10))
val e2 : Expr = FunCall( FunDef("z", Neg(Ident("z"))), True)
val e3 : Expr = FunCall( FunDef("z", Neg(Ident("z"))), Const(30))
val emptyenv : Environment = Map()
assert( eval( e1, emptyenv) == NumVal(20), "test 3")
assert( eval( e2, emptyenv) == BinVal(false), "test 2")
assert( eval( e3, emptyenv) == NumVal(-30), "test 3")
passed(3)


*** Tests Passed (3 points) ***


e1: Expr = FunCall(FunDef("y", Mult(Ident("y"), Const(2))), Const(10))
e2: Expr = FunCall(FunDef("z", Neg(Ident("z"))), True)
e3: Expr = FunCall(FunDef("z", Neg(Ident("z"))), Const(30))
emptyenv: Environment = Map()

In [11]:
val e1 : Expr = Let( "f", 
                     FunDef("x", Plus(Const(1), Ident("x"))),
                     FunCall( Ident("f"), Const(2)))
val e2 : Expr = Let("f",
                   FunDef("x",
                     FunDef("y",
                       Plus(Ident("x"),Ident("y")))),
                   FunCall(FunCall(Ident("f"), Const(2)), Const(3))
                   )
val e3 : Expr = Let("fact", 
                 FunDef("x",
                   ITE( Geq(Const(1), Ident("x")), 
                        Const(1),
                        Mult( Ident("x"),
                              FunCall( Ident("fact"),
                                       Minus(Ident("x"), Const(1)))))),
                 FunCall( Ident("fact"),
                          Const(5))
                   )
val emptyenv : Environment = Map()
assert( eval( e1, emptyenv) == NumVal(3), "test 3")
assert( eval( e2, emptyenv) == NumVal(5), "test 3")
assert( eval( e3, emptyenv) == Error, "test 3")
passed(4)


*** Tests Passed (4 points) ***


e1: Expr = Let(
  "f",
  FunDef("x", Plus(Const(1), Ident("x"))),
  FunCall(Ident("f"), Const(2))
)
e2: Expr = Let(
  "f",
  FunDef("x", FunDef("y", Plus(Ident("x"), Ident("y")))),
  FunCall(FunCall(Ident("f"), Const(2)), Const(3))
)
e3: Expr = Let(
  "fact",
  FunDef(
    "x",
    ITE(
      Geq(Const(1), Ident("x")),
      Const(1),
      Mult(Ident("x"), FunCall(Ident("fact"), Minus(Ident("x"), Const(1))))
    )
  ),
  FunCall(Ident("fact"), Const(5))
)
emptyenv: Environment = Map()

In [12]:
val e1 = Let( "y", Const(10),
           Let( "f", FunDef("x", Plus(Ident("x"),Ident("y"))),
             FunCall(Ident("f"), Const(5))))
val e2 = Let( "f", 
           Let( "y", Const(10),
             FunDef("x", Plus(Ident("x"),Ident("y")))),
           FunCall(Ident("f"), Const(5)))
val e3 = Let( "f", 
           Let( "y", Const(10),
             FunDef("x", Plus(Ident("x"),Ident("y")))),
           Let("y", Const(20), 
               FunCall(Ident("f"), Const(5))))
val e4 = Let( "f", 
             FunDef("x", Plus(Ident("x"),Ident("y"))),
           Let("y", Const(20), 
               FunCall(Ident("f"), Const(5))))
val emptyenv : Environment = Map()
assert( eval( e1, emptyenv) == NumVal(15), "test 1")
assert( eval( e2, emptyenv) == NumVal(15), "test 2")
assert( eval( e3, emptyenv) == NumVal(15), "test 2")
assert( eval( e4, emptyenv) == Error, "test 2")
passed(4)


*** Tests Passed (4 points) ***


e1: Let = Let(
  "y",
  Const(10),
  Let(
    "f",
    FunDef("x", Plus(Ident("x"), Ident("y"))),
    FunCall(Ident("f"), Const(5))
  )
)
e2: Let = Let(
  "f",
  Let("y", Const(10), FunDef("x", Plus(Ident("x"), Ident("y")))),
  FunCall(Ident("f"), Const(5))
)
e3: Let = Let(
  "f",
  Let("y", Const(10), FunDef("x", Plus(Ident("x"), Ident("y")))),
  Let("y", Const(20), FunCall(Ident("f"), Const(5)))
)
e4: Let = Let(
  "f",
  FunDef("x", Plus(Ident("x"), Ident("y"))),
  Let("y", Const(20), FunCall(Ident("f"), Const(5)))
)
emptyenv: Environment = Map()

## Problem 2 (32 points) : Well Formed Expressions

### Well formed epressions

We have seen many expressions in Lettuce that can Error. Ideally, would like to catch catch errors in Lettuce programs at compile time. One class of errors we have seen are errors related to variable liveness.


In the following programs, there is a variable that is used before it is defined. This causes in error when we try to look up that identifier in the environment.


``x + 1``

``let y = 1 in x + y``

``
let f =
  function(x) x + y
in let y = 
  10
in 
  f(20)``

Variables that are *live* are have been defined before they are used and are in scope when they are used. A *Well Formed* expression is one where all the variables are live. All of the above programs are not well formed.

###  Inference rules to define Well Formed Expressions

Let us define inference rules that will help us check if an expression is checked before use. To do so, we will define a set of currently declared variables that are in scope called $S$. 

We will define inference rules for _judgements_ of the form: $WellFormed(\texttt{e}, S)$ that says that expression `e` is _WellFormed_ under the set of declared in scope variables $S$.

Constants are well formed under any set $S$. 

$$ \begin{array}{c}
\\
\hline
WellFormed(\texttt{Const(f)}, S) \\
\end{array} \text{(const-rule)} $$

The rules for `True` and `False` are similar.

$$ \begin{array}{c}
\\
\hline
WellFormed(\texttt{True}, S) \\
\end{array} \text{(true-rule)} $$

$$ \begin{array}{c}
\\
\hline
WellFormed(\texttt{False}, S) \\
\end{array} \text{(false-rule)} $$

Identifiers are well formed only if they belong to $S$:

$$\begin{array}{c}
x \in S \\
\hline
WellFormed(\texttt{Ident(x)}, S) \\
\end{array} \text{(ident-rule)} $$

The rules for binary operator `Plus, Minus, Mult, Geq, Eq, And, Or` are all very similar.

$$ \begin{array}{c}
WellFormed(\texttt{e1}, S) \;\;\; WellFormed(\texttt{e2}, S)\;\;\; T \in \{ \texttt{Plus}, \texttt{Minus}, \texttt{Mult}, \texttt{Geq}, \texttt{Eq}, \texttt{And}, \texttt{Or} \} \\
\hline
WellFormed(\texttt{T(e1, e2)}, S) \\
\end{array} \text{(binary-op)} $$

Likewise, for unary operators 

$$ \begin{array}{c}
WellFormed(\texttt{e1}, S)\\
\hline
WellFormed(\texttt{Neg(e1)}, S) \\
\end{array} \text{(unary-op)} $$

And the ternary if-then-else operator

$$ \begin{array}{c}
WellFormed(\texttt{e1}, S) \;\;\; WellFormed(\texttt{e2}, S)\;\;\; WellFormed(\texttt{e2}, S)\\
\hline
WellFormed(\texttt{ITE(e1, e2, e3)}, S) \\
\end{array} \text{(ternary-op)} $$

The main rule is for let bindings

$$\begin{array}{c}
WellFormed(\texttt{e1}, S) \;\;\; WellFormed(\texttt{e2}, S \cup \{ x\} ) \\
\hline
WellFormed(\texttt{Let(x, e1, e2)}, S) \\
\end{array} \text{(let-rule)} $$

Suppose `e1` is well formed under $S$ and `e2` is well formed under $S$ with the identifier `x` added,
then `let x = e1 in e2` is well formed under $S$, as well. 
- This is confusing when you first see it, but repeat the sentence above and match it with the rule, 
- Notice that the defining expression must be well formed under the original S but the body expression is allowed to be well formed under $S \cup \{ x\} $.

We can also add the rules for function definition:

$$\begin{array}{c}
WellFormed(\texttt{e}, S \cup \{x\}) \\
\hline
WellFormed(\texttt{FunDef(x, e)}, S ) \\
\end{array} \; \text{(fundef-rule)} $$

Finally, function calls are very simple:

$$ \begin{array}{c}
\texttt{f} \in S,\;\;\;\; WellFormed(\texttt{e}, S) \\
\hline
WellFormed(\texttt{f(e)}, S ) \\
\end{array} \text{(well-formed-funcall)} $$

The name of the function `f` better be defined in the scope $S$ and the argument better be OK. Then
we can say that the function call is.

### Problem 2a

Write a function that determines whether a lettuce expression is well formed based on the inference rules above.

This problem does not have any tests, it will be tested by problem 1b.

In [13]:
def wellFormed(e: Expr, S: Set[String]): Boolean = e match {
  case Const(_)  => true
  case True => true
  case False => true
  case Ident(x) => S.contains(x)
  case Plus(e1, e2) => wellFormed(e1, S) && wellFormed(e2, S)
  case Minus(e1, e2) => wellFormed(e1, S) && wellFormed(e2, S)
  case Mult(e1, e2) => wellFormed(e1, S) && wellFormed(e2, S)
  case Geq(e1, e2) => wellFormed(e1, S) && wellFormed(e2, S)
  case Eq(e1, e2) => wellFormed(e1, S) && wellFormed(e2, S)
  case And(e1, e2) => wellFormed(e1, S) && wellFormed(e2, S)
  case Or(e1, e2) => wellFormed(e1, S) && wellFormed(e2, S)
  case Neg(e) => wellFormed(e, S)
  case ITE(e1, e2, e3) => wellFormed(e1, S) && wellFormed(e2, S) && wellFormed(e3, S)
  case Let(x, e1, e2) => wellFormed(e1, S) && wellFormed(e2, S + x)
  case FunDef(x, e) => wellFormed(e, S + x)
  case FunCall(f, e) => wellFormed(f, S) && wellFormed(e, S)
}

defined function wellFormed

### Well Formed Lettuce Programs

Recall that a Lettuce program is a lettuce expression that starts executing in an empty environment with no variables declared

$$\mathbf{Program} \rightarrow Main(\mathbf{Expr}) $$

We say that a lettuce program is well formed if its expressions is well formed when no variables are in scope initially. We can make a judgement that says a program `p` is well formed: $WellFormedProgram(\texttt{p})$. And we can define this judgement using inference rules:


$$\begin{array}{c}
WellFormed(\texttt{e}, \emptyset) \\
\hline
WellFormedProgram(\texttt{Main(e)}) \\
\end{array} \text{(wf-program)} $$



In [14]:
sealed trait Program
case class Main(p : Expr) extends Program

defined trait Program
defined class Main

### Problem 2b (32 points)

Write a function ``WF`` that determines whether a Lettuce program is well formed based on the inference rules above.

$$ WF : Program \rightarrow Boolean$$

In [15]:
def WF(main: Main) = main match {
  case Main(e: Expr) => wellFormed(e, Set())
  case _ => false
}

defined function WF

In [16]:
val p1 = Main( Const(0) )
val p2 = Main( Const(10) )
val p3 = Main( True )
val p4 = Main( False )
assert( WF(p1), "test 1")
assert( WF(p2), "test 2")
assert( WF(p3), "test 3")
assert( WF(p4), "test 4")
passed(2)


*** Tests Passed (2 points) ***


p1: Main = Main(Const(0))
p2: Main = Main(Const(10))
p3: Main = Main(True)
p4: Main = Main(False)

In [17]:
val p1 = Main( Ident("x") )
val p2 = Main( Ident("y") )
assert( !WF(p1), "test 1")
assert( !WF(p2), "test 2")
passed(2)


*** Tests Passed (2 points) ***


p1: Main = Main(Ident("x"))
p2: Main = Main(Ident("y"))

In [18]:
val p1 = Main( Plus( Const(10), Const(20)) )
val p2 = Main( Minus( Ident("x"), Ident("x")) )
val p3 = Main( Mult(Plus( Const(10), True), Mult( Const(30), Const(40))) )
val p4 = Main( Mult(Plus( Const(10), Const(20)), Mult( Const(30), Ident("y"))) )
assert(  WF(p1), "test 1")
assert( !WF(p2), "test 2")
assert(  WF(p3), "test 3")
assert( !WF(p4), "test 4")
passed(1)


*** Tests Passed (1 point) ***


p1: Main = Main(Plus(Const(10), Const(20)))
p2: Main = Main(Minus(Ident("x"), Ident("x")))
p3: Main = Main(Mult(Plus(Const(10), True), Mult(Const(30), Const(40))))
p4: Main = Main(Mult(Plus(Const(10), Const(20)), Mult(Const(30), Ident("y"))))

In [19]:
val p1 = Main( And(And(True, True), And(False,False)) )
val p2 = Main( Or( Or(True, Ident("b1")), Or(Ident("b2"), True)) )
val p3 = Main( And( And( Const(10), True), Or(False, Const(0))) )
assert(  WF(p1), "test 1")
assert( !WF(p2), "test 2")
assert(  WF(p3), "test 3")
passed(1)


*** Tests Passed (1 point) ***


p1: Main = Main(And(And(True, True), And(False, False)))
p2: Main = Main(Or(Or(True, Ident("b1")), Or(Ident("b2"), True)))
p3: Main = Main(And(And(Const(10), True), Or(False, Const(0))))

In [20]:
val p1 = Main( Geq(Const(2), Const(1)))
val p2 = Main( Geq( Plus(Const(1), True), False))
val p3 = Main( Eq( Geq(Const(2), Const(1)), Plus(Const(3), Const(3))))
val p4 = Main( Eq( Geq(Const(2), Const(1)), Plus(Ident("var"), Const(3))))
assert(  WF(p1), "test 1")
assert(  WF(p2), "test 2")
assert(  WF(p3), "test 3")
assert( !WF(p4), "test 4")
passed(1)


*** Tests Passed (1 point) ***


p1: Main = Main(Geq(Const(2), Const(1)))
p2: Main = Main(Geq(Plus(Const(1), True), False))
p3: Main = Main(Eq(Geq(Const(2), Const(1)), Plus(Const(3), Const(3))))
p4: Main = Main(Eq(Geq(Const(2), Const(1)), Plus(Ident("var"), Const(3))))

In [21]:
val p1 = Main( Neg(True) )
val p2 = Main( Neg(Neg(Neg(Neg(Neg(Neg(Const(1))))))) )
val p3 = Main( Neg(Neg(Ident("Neg"))) )
assert(  WF(p1), "test 1")
assert(  WF(p2), "test 2")
assert( !WF(p3), "test 3")
passed(2)


*** Tests Passed (2 points) ***


p1: Main = Main(Neg(True))
p2: Main = Main(Neg(Neg(Neg(Neg(Neg(Neg(Const(1))))))))
p3: Main = Main(Neg(Neg(Ident("Neg"))))

In [22]:
val p1 = Main( ITE(Const(1), Const(2), Const(3)) )
val p2 = Main( ITE(True, Const(1), Ident("f")) )
val p3 = Main( ITE(False, Ident("t"), Const(1)) )
assert(  WF(p1), "test 1")
assert( !WF(p2), "test 2")
assert( !WF(p3), "test 3")
passed(3)


*** Tests Passed (3 points) ***


p1: Main = Main(ITE(Const(1), Const(2), Const(3)))
p2: Main = Main(ITE(True, Const(1), Ident("f")))
p3: Main = Main(ITE(False, Ident("t"), Const(1)))

In [23]:
val p1 = Main(Let( "x", Const(1), Const(1)) )
val p2 = Main(Let( "x", Const(1), Ident("x")) )
val p3 = Main(Let( "x", Plus(Const(1), Const(1)), Plus(Ident("x"), Const(1))) )
val p4 = Main(Let( "x", Ident("x"), Const(1)) )
val p5 = Main(Let( "x", Const(1), Plus( Ident("x"),Ident("y"))) )
assert(  WF(p1), "test 1")
assert(  WF(p2), "test 2")
assert(  WF(p3), "test 3")
assert( !WF(p4), "test 4")
assert( !WF(p5), "test 5")
passed(2)


*** Tests Passed (2 points) ***


p1: Main = Main(Let("x", Const(1), Const(1)))
p2: Main = Main(Let("x", Const(1), Ident("x")))
p3: Main = Main(Let("x", Plus(Const(1), Const(1)), Plus(Ident("x"), Const(1))))
p4: Main = Main(Let("x", Ident("x"), Const(1)))
p5: Main = Main(Let("x", Const(1), Plus(Ident("x"), Ident("y"))))

In [24]:
val p1 = Main( Let("x", Const(1), 
                 Let("y", Const(2), 
                   Let("z", Const(3), 
                     Plus( Plus(Ident("x"), Ident("y")), Plus(Ident("z"), Const(4)))))) )
val p2 = Main( Let("x", 
                   Let("y", Const(1), Ident("y")), 
                   Plus(Const(1), Ident("x"))) )
val p3 = Main( Let("x", 
                   Let("y", Const(1), Ident("y")), 
                   Plus(Ident("y"), Ident("x"))) )
assert(  WF(p1), "test 1")
assert(  WF(p2), "test 2")
assert( !WF(p3), "test 3")
passed(2)


*** Tests Passed (2 points) ***


p1: Main = Main(
  Let(
    "x",
    Const(1),
    Let(
      "y",
      Const(2),
      Let(
        "z",
        Const(3),
        Plus(Plus(Ident("x"), Ident("y")), Plus(Ident("z"), Const(4)))
      )
    )
  )
)
p2: Main = Main(
  Let("x", Let("y", Const(1), Ident("y")), Plus(Const(1), Ident("x")))
)
p3: Main = Main(
  Let("x", Let("y", Const(1), Ident("y")), Plus(Ident("y"), Ident("x")))
)

In [25]:
val p1 = Main( FunDef("a", Const(1)) )
val p2 = Main( FunDef("b", Ident("b")) )
val p3 = Main( FunDef("c", Ident("d")) )
val p4 = Main( FunDef("e", Mult( Ident("e"), Ident("e"))) )
val p5 = Main( FunDef("f", Eq( Ident("f"), Ident("g"))) )
val p6 = Main( FunDef("g", Let("h", Const(1), Mult(Ident("g"), Ident("h")))) )
assert(  WF(p1), "test 1")
assert(  WF(p2), "test 2")
assert( !WF(p3), "test 3")
assert(  WF(p4), "test 4")
assert( !WF(p5), "test 5")
assert(  WF(p6), "test 6")
passed(4)


*** Tests Passed (4 points) ***


p1: Main = Main(FunDef("a", Const(1)))
p2: Main = Main(FunDef("b", Ident("b")))
p3: Main = Main(FunDef("c", Ident("d")))
p4: Main = Main(FunDef("e", Mult(Ident("e"), Ident("e"))))
p5: Main = Main(FunDef("f", Eq(Ident("f"), Ident("g"))))
p6: Main = Main(FunDef("g", Let("h", Const(1), Mult(Ident("g"), Ident("h")))))

In [26]:
val p1 = Main( FunCall(FunDef("a", Const(1)), Const(1)) )
val p2 = Main( FunCall(FunDef("b", Const(1)), Ident("b")) )
val p3 = Main( FunCall(FunDef("c", Ident("d")), Const(1)) )
assert(  WF(p1), "test 1")
assert( !WF(p2), "test 2")
assert( !WF(p3), "test 3")
passed(4)


*** Tests Passed (4 points) ***


p1: Main = Main(FunCall(FunDef("a", Const(1)), Const(1)))
p2: Main = Main(FunCall(FunDef("b", Const(1)), Ident("b")))
p3: Main = Main(FunCall(FunDef("c", Ident("d")), Const(1)))

In [27]:
val p1 = Main( Let("fact", 
                 FunDef("x",
                   ITE( Geq(Const(1), Ident("x")), 
                        Const(1),
                        Mult( Ident("x"),
                              FunCall( Ident("fact"),
                                       Minus(Ident("x"), Const(1)))))),
                 FunCall( Ident("fact"),
                          Const(5))
                   )
             )
assert( !WF(p1) )
passed(4)


*** Tests Passed (4 points) ***


p1: Main = Main(
  Let(
    "fact",
    FunDef(
      "x",
      ITE(
        Geq(Const(1), Ident("x")),
        Const(1),
        Mult(Ident("x"), FunCall(Ident("fact"), Minus(Ident("x"), Const(1))))
      )
    ),
    FunCall(Ident("fact"), Const(5))
  )
)

In [28]:
val p1 = Main( Let("f",
                   FunDef("x",
                     FunDef("y",
                       Plus(Ident("x"),Ident("y")))),
                   FunCall(FunCall(Ident("f"), Const(2)), Const(3))
                   )
              )
assert(  WF(p1) )
passed(4)


*** Tests Passed (4 points) ***


p1: Main = Main(
  Let(
    "f",
    FunDef("x", FunDef("y", Plus(Ident("x"), Ident("y")))),
    FunCall(FunCall(Ident("f"), Const(2)), Const(3))
  )
)